In [ ]:
import pandas as pd
import pandas_datareader.data as pdr
from datetime import datetime
from datetime import date
from datetime import timedelta
import math

In [ ]:
# Step 1: Get and organize data
# Step 2: Run algorithm
# Step 3: Calculate/Analyze Return vs Benchmark(s)
# Step 4: Test Inputs/Tweak Strategy

In [ ]:
def cagr(_df):
    df = _df.copy()
    df = df.sort_index(ascending=True)
    df["cum_return"] = (1 + df["return"]).cumprod()
    total_ret = df['cum_return'][-1]
    start = datetime.utcfromtimestamp(df.index.values[0].astype('O')/1e9)
    end = datetime.utcfromtimestamp(df.index.values[-1].astype('O')/1e9)
    period_years = (end - start).days / 365.25
    CAGR = (total_ret)**(1/period_years) - 1
    return CAGR

In [ ]:
def volatility(_df):
    start = datetime.utcfromtimestamp(_df.index.values[0].astype('O')/1e9)
    end = datetime.utcfromtimestamp(_df.index.values[-1].astype('O')/1e9)
    period_years = (end - start).days / 365.25
    ratio_to_annual = _df['return'].count() / period_years
    vol = _df["return"].std() * math.sqrt(ratio_to_annual)
    return vol

In [ ]:
def sharpe(_df, risk_free_rate):
    ret = cagr(_df)
    vol = volatility(_df)
    sharpe = (ret - risk_free_rate) / vol
    return sharpe

In [ ]:
def max_drawdown(_df):
    df = _df.copy()
    df = df.sort_index(ascending=True)
    df["cum_return"] = (1 + df["return"]).cumprod()
    df["cum_return_max"] = df["cum_return"].cummax()
    df["drawdown"] = df["cum_return_max"] - df["cum_return"]
    df["drawdown_pct"] = df["drawdown"] / df["cum_return_max"]
    max_dd = df.loc[df["drawdown_pct"].idxmax()]
    return {
        'drawdown': max_dd['drawdown_pct'],
        'date': max_dd.name
    }

In [ ]:
# Step 1: Get and organize data

start = datetime(1900, 1, 1)
end = datetime.now()
spy_daily = pdr.get_data_yahoo('SPY',start,end,interval='d')
spy_daily = spy_daily[['Adj Close']].rename(columns={'Adj Close': 'price'})
spy_daily

In [ ]:
spy_actions = pdr.DataReader('SPY', 'yahoo-actions', start, end)
spy_dividends = spy_actions[spy_actions['action'] == 'DIVIDEND']
spy_dividends

In [ ]:
spy_dividends = spy_dividends[['value']].rename(columns={'value': 'dividend'})
spy_dividends

In [ ]:
spy = spy_daily.merge(spy_dividends, left_index=True, right_index=True, how='outer')
spy

In [ ]:
spy[-70:-45]

In [ ]:
spy['dividend'] = spy['dividend'].fillna(0)
spy[-70:-55]

In [ ]:
spy['price_change'] = spy['price'].pct_change()
spy

In [ ]:
spy['dividend_yield'] = spy['dividend'] / spy['price']
spy

In [ ]:
spy[-80:-55]

In [ ]:
spy['return'] = spy['dividend_yield'] + spy['price_change']
spy

In [ ]:
cagr(spy)

In [ ]:
def get_returns_for_asset(symbol, start = datetime(1900, 1, 1), end = datetime.now()):
    daily = pdr.get_data_yahoo(symbol,start,end,interval='d')
    daily = daily[['Adj Close']].rename(columns={'Adj Close': 'price'})
    actions = pdr.DataReader(symbol, 'yahoo-actions', start, end)
    dividends = actions[actions['action'] == 'DIVIDEND']
    dividends = dividends[['value']].rename(columns={'value': 'dividend'})
    df = daily.merge(dividends, left_index=True, right_index=True, how='outer')
    df['dividend'] = df['dividend'].fillna(0)
    df['price_change'] = df['price'].pct_change()
    df['dividend_yield'] = df['dividend'] / df['price']
    df['return'] = df['dividend_yield'] + df['price_change']
    return df

In [ ]:
portfolio = [
    {
        'symbol': 'SPY', # Stocks
        'weight': 0.3
    },
    {
        'symbol': 'TLT', # Long Term Bond
        'weight': 0.4
    },
    {
        'symbol': 'IEF', # Intermediate Term Bond
        'weight': 0.15
    },
    {
        'symbol': 'GLD', # Gold
        'weight': 0.075
    },
    {
        'symbol': 'DJP', # Commidities
        'weight': 0.075
    }
]

In [ ]:
for asset in portfolio:
    returns = get_returns_for_asset(asset['symbol'])
    asset['returns'] = returns
    
portfolio

In [ ]:
portfolio_df = pd.DataFrame()

for asset in portfolio:
    returns = asset['returns']
    for column in returns.columns:
        column_name = "{}_{}".format(column, asset['symbol'])
        print(column_name)
        portfolio_df[column_name] = returns[column]
            
        
portfolio_df = portfolio_df.dropna()
portfolio_df

In [ ]:
# Step 2: Run algorithm

def get_returns_for_portfolio(_portfolio, _df):
    df = _df.copy()
    running_values = {}

    total_value = previous_total = starting_value = 10000
    for x in df.index:  
        ser = df.loc[x]

        # Calculate return
        for asset in _portfolio:
            symbol = asset['symbol']
            col_name_price = "price_{}".format(symbol)
            col_name_qty = "quantity_{}".format(symbol)
            col_name_value = "value_{}".format(symbol)
            col_name_dividend = "dividend_{}".format(symbol)

            if symbol in running_values and 'qty' in running_values[symbol]:
                qty = running_values[symbol]['qty']
            else:
                qty = 0

            asset_price = ser[col_name_price]
            asset_value = qty * asset_price
            asset_dividend = qty * ser[col_name_dividend]
            total_value += asset_value + asset_dividend

            df.loc[x, col_name_qty] = qty
            df.loc[x, col_name_value] = asset_value

        # Calculate reinvestment
        for asset in _portfolio:
            symbol = asset['symbol']
            col_name_price = "price_{}".format(symbol)
            col_name_req_amt = "required_amount_{}".format(symbol)
            col_name_req_qty = "required_quantity_{}".format(symbol)

            asset_price = ser[col_name_price]
            req_amt = asset['weight'] * total_value
            req_qty = req_amt / asset_price

            df.loc[x, col_name_req_amt] = req_amt
            df.loc[x, col_name_req_qty] = req_qty

            if symbol not in running_values:
                running_values[symbol] = {}
            running_values[symbol]['qty'] = req_qty

        df.loc[x, 'total_value'] = total_value
        previous_total = total_value
        total_value = 0
        
    df['return'] = df['total_value'].pct_change()
    return df


all_weather_df = get_returns_for_portfolio(portfolio, portfolio_df)

In [ ]:
all_weather_df

In [ ]:
cagr(all_weather_df)

In [ ]:
volatility(all_weather_df)

In [ ]:
sharpe(all_weather_df, 0.02)

In [ ]:
max_drawdown(all_weather_df)

In [ ]:
# Step 3: Calculate/Analyze Return vs Benchmark(s)

portfolio_only_spy = [
    {
        'symbol': 'SPY',
        'weight': 1
    }
]

only_spy_df = get_returns_for_portfolio(portfolio_only_spy, portfolio_df)

In [ ]:
cagr(only_spy_df)

In [ ]:
volatility(only_spy_df)

In [ ]:
sharpe(only_spy_df, 0.02)

In [ ]:
max_drawdown(only_spy_df)

In [ ]:
frame = { 
    'All Weather': all_weather_df['total_value'], 
    'Only SPY': only_spy_df['total_value']
} 
  
graph_df = pd.DataFrame(frame) 
graph_df.plot(figsize=(15, 6))

In [ ]:
# Step 4: Test Inputs/Tweak Strategy

In [ ]:
portfolio_2 = [
    {
        'symbol': 'SPY', # S&P 500
        'weight': 0.6
    },
    {
        'symbol': 'TLT', # Long term bonds
        'weight': 0.4
    },
]

for asset in portfolio_2:
    returns = get_returns_for_asset(asset['symbol'])
    asset['returns'] = returns
    

portfolio_2_df = pd.DataFrame()

for asset in portfolio_2:
    returns = asset['returns']
    for column in returns.columns:
        column_name = "{}_{}".format(column, asset['symbol'])
        print(column_name)
        portfolio_2_df[column_name] = returns[column]
            
        
portfolio_2_df = portfolio_2_df.dropna()
portfolio_2_df

In [ ]:
portfolio_2_df = portfolio_2_df['2006-10-31':]
portfolio_2_df

In [ ]:
portfolio_2_df = get_returns_for_portfolio(portfolio_2, portfolio_2_df)
portfolio_2_df

In [ ]:
cagr(portfolio_2_df)

In [ ]:
volatility(portfolio_2_df)

In [ ]:
sharpe(portfolio_2_df, 0.02)

In [ ]:
max_drawdown(portfolio_2_df)

In [ ]:
frame = { 
    '60/40': portfolio_2_df['total_value'], 
    'All Weather': all_weather_df['total_value'], 
    'Only SPY': only_spy_df['total_value']
} 
  
graph_df = pd.DataFrame(frame) 
graph_df.plot(figsize=(15, 6))